In [1]:
import numpy as np
import os
import sys
from pathlib import Path

module_path = str(Path(os.path.abspath('')).parent)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.sac_modular import *
from functions.RNCRN_tools import * 

In [2]:
# creates the classification-controlled RNCRN from the two subsystem models i.e. point-wise parametrized RNCRN & Quasi-static classification CRN
# set initial conditions for CRN (overwritten later)
env_inits = [0.5, 0.5]
neuron_classify_inits = [0, 0, 0, 0]
bif_inits = [0]
exec_species = [2.5]
neuron_toggle_inits = [0, 0, 0]

# set timescale for both subsystems
time_scale_classify = 0.001
time_scale_toggle = 0.001

# filenames of subsystem models 
filename_classify = r'models/example_classify_subsystem'
filename_toggle = r'models/example_param_RNCRN'

# output location of the compiled classification-controlled RNCRN (as a CRN)
crn_file = 'CRNs/xor_uni_bi_CRN.txt'

# function to compile models into a chemical reaction network 
CRN = create_sense_and_respond_CRN(filename_classify, filename_toggle, time_scale_classify, time_scale_toggle , env_inits, neuron_classify_inits, bif_inits,exec_species, neuron_toggle_inits)
save_crn(crn_file, CRN )
print(CRN)

# loads the chemical reaction network file from .txt and extracts the kinetic and stoichiometric data
species, reaction_rates, react_stoch, prod_stoch, stoch_mat, number_species, number_reactions, initial_concs = read_crn_txt(crn_file)

# creates a tuple of kinetic and stoichiometric data for the trajectory simulation 
args_tup_rncrn = (reaction_rates, react_stoch, stoch_mat,)

#L_1=0.5,L_2=0.5,Z_1=0,Z_2=0,Z_3=0,Z_4=0,R_1=0
L_1 + Z_1->L_1 + Z_1 + Z_1,2725.7585525512695
L_1 + Z_2->L_1 + Z_2 + Z_2,2258.833169937134
L_1 + Z_3->L_1,50.66689848899841
L_1 + Z_4->L_1 + Z_4 + Z_4,157.9049676656723
L_2 + Z_1->L_2,3025.1846313476562
L_2 + Z_2->L_2 + Z_2 + Z_2,57.44681879878044
L_2 + Z_3->L_2,2511.180877685547
L_2 + Z_4->L_2,156.2095284461975
Z_1->Z_1 + Z_1,296.2667644023895
Z_2->,2328.4013271331787
Z_3->Z_3 + Z_3,2581.545352935791
Z_4->Z_4 + Z_4,665.9629344940186
->Z_1,10.0
->Z_2,10.0
->Z_3,10.0
->Z_4,10.0
Z_1 + Z_1->Z_1,1000.0
Z_2 + Z_2->Z_2,1000.0
Z_3 + Z_3->Z_3,1000.0
Z_4 + Z_4->Z_4,1000.0
Z_1 + R_1->Z_1 + R_1 + R_1,4527.679443359375
Z_2 + R_1->Z_2,5719.223976135254
Z_3 + R_1->Z_3,5648.273944854736
Z_4 + R_1->Z_4 + R_1 + R_1,777.0170569419861
R_1->R_1 + R_1,682.2481751441956
->R_1,10.0
R_1 + R_1->R_1,1000.0
R_1 + Y_1->R_1,477660.9191894531
R_1 + Y_2->R_1,21494744.140625
R_1 + Y_3->R_1,1876260.7421875
#X_1=2.5,Y_1=0,Y_2=0,Y_3=0
X_1 + Y_1->Y_1,54.40314
X_1 + Y_2->Y_2 

In [3]:
# numerical integration of the RNCRN 
# creates a grid of (lambda_1, Lambda_2) pairs for numerical integration of full classification-controlled RNCRN
vals = np.arange(0, 2.1, 0.1)
num_graphs = len(vals)
row = 0
column = 0
xv, yv = np.meshgrid(vals, np.flip(vals))
xval = xv.flatten()
yval = yv.flatten()

# the time length of the trajectory
tFinal = 1

for i in np.arange(1, num_graphs*num_graphs + 1, 1):
    print(i)
    lambda_1 = xval[i-1]
    lambda_2 = yval[i-1]
    
    # checks if data already created. If not, then runs numerical integration
    # does this for two initial conditions ('low' and 'high')
    try:
        t_low = np.load('data/sol_rncrn_low_t_i'+str(i)+'.npy')
        y_low = np.load('data/sol_rncrn_low_y_i'+str(i)+'.npy')
    except:
        inits_rncrn_low, id_dict_low = new_initial_conditions(initial_concs, species, {'L_1':lambda_1, 'L_2': lambda_2, 'X_1': 2.0})
        sol_rncrn_low = solve_ivp(stoch_mat_to_mass_action, [0, tFinal], inits_rncrn_low, args=args_tup_rncrn, rtol=10e-8)
        np.save('data/sol_rncrn_low_t_i'+str(i)+'.npy', sol_rncrn_low.t)
        np.save('data/sol_rncrn_low_y_i'+str(i)+'.npy', sol_rncrn_low.y[id_dict_low['X_1'],:])
    try: 
        t_high = np.load('data/sol_rncrn_high_t_i'+str(i)+'.npy')
        y_high = np.load('data/sol_rncrn_high_y_i'+str(i)+'.npy')
    except:
        inits_rncrn_high, id_dict_high = new_initial_conditions(initial_concs, species, {'L_1':lambda_1, 'L_2': lambda_2, 'X_1': 7.0})
        sol_rncrn_high = solve_ivp(stoch_mat_to_mass_action, [0, tFinal], inits_rncrn_high, args=args_tup_rncrn, rtol=10e-8)    
        np.save('data/sol_rncrn_high_t_i'+str(i)+'.npy', sol_rncrn_high.t)
        np.save('data/sol_rncrn_high_y_i'+str(i)+'.npy', sol_rncrn_high.y[id_dict_high['X_1'],:])
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
